In [2]:
from transformers import BartForConditionalGeneration, AutoTokenizer
bart = BartForConditionalGeneration.from_pretrained('hyunwoongko/kobart')
tokenizer = AutoTokenizer.from_pretrained('hyunwoongko/kobart')

In [3]:
texts = ['최근 들어 딥러닝', '안녕 친구야']
sample_ids = tokenizer(texts, padding=True, return_tensors='pt')['input_ids']
sample_ids

tensor([[19261, 14301,  1700, 10021, 10277,  9747],
        [11699,  9592, 15230, 11734,     3,     3]])

In [4]:
outs = bart.generate(
    sample_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences=1,
    early_stopping=True,
)

In [31]:
import torch
import torch.nn.functional as F

def rerank(matrix, repeat_time, margin=0.):
    upper = matrix.unsqueeze(-1).repeat_interleave(repeat_time, dim=-1)
    lower = matrix.unsqueeze(1).repeat_interleave(repeat_time, dim=1)
    return F.relu((lower - upper + margin).triu(1))

In [32]:
batch_size = 8
num_cands = 16
# # embed_size = docs['input_ids'].size(-1)
# assert num_cands == cands['input_ids'].size(0) // batch_size

# doc_embeddings = self.encoder(**docs)[0][:, 0, :]
# gold_embeddings = self.encoder(**golds)[0][:, 0, :]
# sims_doc_gold = torch.cosine_similarity(doc_embeddings, gold_embeddings, dim=-1)

# cand_embeddings = self.encoder(**cands)[0][:, 0, :].view(batch_size, num_cands, -1)
# doc_embeddings = doc_embeddings.unsqueeze(1).repeat_interleave(num_cands, dim=1)
# sims_doc_cand = torch.cosine_similarity(doc_embeddings, cand_embeddings, dim=-1)

# scores_gold = sims_doc_cand - sims_doc_gold.unsqueeze(1).repeat_interleave(num_cands, dim=1)
# scores_gold = F.relu(scores_gold).sum(1)

ranks = (
    torch.arange(num_cands)
    .unsqueeze(0)
    .repeat_interleave(batch_size, dim=0)
)
rerank(ranks, num_cands)

tensor([[[ 0.,  1.,  2.,  ..., 13., 14., 15.],
         [ 0.,  0.,  1.,  ..., 12., 13., 14.],
         [ 0.,  0.,  0.,  ..., 11., 12., 13.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  1.,  2.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  1.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[ 0.,  1.,  2.,  ..., 13., 14., 15.],
         [ 0.,  0.,  1.,  ..., 12., 13., 14.],
         [ 0.,  0.,  0.,  ..., 11., 12., 13.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  1.,  2.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  1.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[ 0.,  1.,  2.,  ..., 13., 14., 15.],
         [ 0.,  0.,  1.,  ..., 12., 13., 14.],
         [ 0.,  0.,  0.,  ..., 11., 12., 13.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  1.,  2.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  1.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        ...,

        [[ 0.,  1.,  2.,  ..., 13., 14., 15.],
         [ 0.,  0.,  1.,  ..., 12., 13., 14.]

In [33]:
import pandas as pd
pd.DataFrame({'ab': [[1,2,3],[1,2,3]]})

,ab
0,"[1, 2, 3]"
1,"[1, 2, 3]"


In [34]:
from datasets import load_metric
rouge = load_metric('rouge')

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [47]:
predictions = ['abc ccc', 'edf']
references = ['abc ccc', 'edf']
# ['최근 들어 딥러닝은 음성, 시각, 자연어, 로봇, 게임 등 인공지능 여러 분야의', '환경을 근본적으로 변화시키고 있다.']
# references = ['최근 들어 딥러닝은 음성, 시각, 자연어, 로봇, 게임 등 인공지능 여러 분야의', '환경을 근본적으로 변화시키고 있다.']
scores = rouge.compute(predictions=predictions, references=references[::-1])

In [48]:
scores['rouge1'].mid.fmeasure

0.0

In [1]:
valid_path = '/Volumes/GoogleDrive/My Drive/gassum/data/valid_w_cands.csv'

In [2]:
import pandas as pd

In [3]:
valid = pd.read_csv(valid_path)

In [5]:
valid = valid.iloc[:, 2:]

In [8]:
valid

,id,title,text,summary,publish_date,size,char_count,category,media_type,media_sub_type,media_name,candidates
0,340626877,"文대통령 ""5G는 4차산업혁명 시대의 고속도로""","[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, 체...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...,2019-04-08 17:43:00,medium,1111,정치,online,경제지,한국경제,"['문 대통령은 8일 서울 올림픽공원에서 열린 5G플러스 전략발표에 참석해""5G 시..."
1,340626896,"""손학규 물러나라""...바른계, 최고위 회의 '보이콧'",] 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다. 바른미래당이 8일 연 ...,8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...,2019-04-08 17:41:00,small,904,종합,online,경제지,한국경제,['바른미래당이 8일 연 최고위원 회의에는 7명의 최고위원 중 손학규 대표와 김관영...
2,340626904,"""마치 한 장소처럼"" ... 5G로 서울·부산·광주 원격협연",[ 홍윤정 기자 ] 8일 서울 올림픽공원 K아트홀. 지난 3일 한국이 세계 최초로 ...,지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...,2019-04-08 17:44:00,medium,1104,"IT,과학",online,경제지,한국경제,"[""지난 3일 한국이 세계 최초로 5세대(5G) 이동통신 서비스를 상용화한 것을 기..."
3,340627450,"""이웃도 모르는데, 이게 서울 미래냐""...'고층 재개발 요구' 작심 비판한 박원순",] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소리...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...,2019-04-08 17:48:00,small,740,사회,online,경제지,한국경제,['박원순 서울시장이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소리를 쏟아...
4,340627465,"""SK·알파벳 벤치마킹해야""...KB운용, KMH에 주주서한","[ 임근호 기자 ] ""SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조를 ...",주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...,2019-04-08 17:49:00,medium,1389,경제,online,경제지,한국경제,['KB운용은 KMH 지분 10.24%를 보유한 주요 주주인 SK(주)와 미국 알파...
...,...,...,...,...,...,...,...,...,...,...,...,...
30117,350851474,"영주시,외국인근로자 관광명소 문화탐방... 한국생활 정착도와",영주시는 이억만리에서 건너온 계절근로자들의 향수를 달래고 안정된 한국생활 적응을 돕...,"영주시는 외국인 근로자들의 향수를 달래고, 안정적인 한국생활 적응을 지원하기 위해 ...",2019-06-30 20:46:00,small,700,종합,online,경제지,헤럴드경제,['영주시는 이억만리에서 건너온 계절근로자들의 향수를 달래고 안정된 한국생활 적응을...
30118,350851925,달 탐사 50주년 기념...전국 과학관에서 우주특별전 열린다,여름 방학을 맞아 전국의 국립과학관에서 달 탐사 50주년과 국제천문연맹(IAU) 설...,달 탐사 50주년과 국제천문연맹(IAU)설립 100주년 기념하는 특별전시가 전국의 ...,2019-06-30 20:59:00,small,706,경제,online,경제지,헤럴드경제,"[""달 탐사 50주년과 국제천문연맹(IAU) 설립 100주년을 기념하는 특별전시가 ..."
30119,350854748,황교안대표 영주서 자유한국당 영주.문경예천당원협의회 시국강연 가져 ....내년총선 ...,영주문경예천 당원협의회 시국강연 및 당원교육에 참석차 영주를 방문한 황교안(오른쪽)...,황교안 대표는 지난 29일 자유한국당 대표로 경북 영주문경예청 당원협의회 당원교육에...,2019-06-30 21:53:00,medium,1221,종합,online,경제지,헤럴드경제,['황교안 자유한국당 대표가 29일 경북영주시를 찾아 김광림 최고위원과 영주시민회관...
30120,350857648,"예천군,양질의 일자리 창출 총력....사회적 경제 기업 활성화에 두팔걷어",경북예천군은 장기적인 국내 경기침체가 지속됨에 따라 사회적경제 기업 육성 등을 통한...,경북예천군은 사회적경제 기업 육성 등을 통한 일자리 창출에 행정동력을 집중한 결과 ...,2019-06-30 22:14:00,small,817,종합,online,경제지,헤럴드경제,['경북예천군은 장기적인 국내 경기침체가 지속됨에 따라 사회적경제 기업 육성 등을 ...
